In [ ]:
# ============================================
# PSEUDO-CODE — Pipeline ET_fractal (tardif)
# Données: SN Ia (Pantheon+), CC (Hz), BAO (DESI 2024)
# Modèle: H(z) = H0 * E(z), avec E(z) piloté par H_D(z)
# Option: H_D(z) = sigmoid(z; params) + bump(z; params) + gating(z; params)
# ============================================

INPUTS:
  SN_catalog: {z_i, mu_i, C_mu}                  # covariance complète
  CC_table:   {z_j, H_j, sigmaH_j}               # diag ou cov
  BAO_vec:    {z_k, y_k, C_bao}                  # y = (DM/rs, DH/rs, DV/rs, ...)
  settings:   priors, bounds, pivot z_pivot=1.037 (si utilisé), integration grid, etc.

PARAMETERS (example):
  theta = {
    # géométrie / cinématique
    params_sigmoid: (HD_low, HD_high, z_t, xi, ...)
    params_bump:    (A_b, z_b, sigma_b, ...)     # amplitude, centre, largeur
    params_gating:  (g0, z_g, w_g, ...)          # coupe/atténuation du bump
    # échelles
    H0, rs_eff, (option: MB ou nuisance SN)
    # nuisances/offsets (selon le traitement)
    nuisance_SN, nuisance_CC, nuisance_BAO
  }

CORE FUNCTIONS:
  HD_of_z(z, theta):
    return sigmoid(z; params_sigmoid) + gating(z)*bump(z; params_bump)

  E_of_z(z, theta):
    # dépend de votre loi cinématique (ex: (1+z)^(6/HD(z)) ou autre forme écrite dans l'étude)
    return E_model(z, HD_of_z(z), theta)

  comoving_distance(z, theta):
    return ∫_0^z dz'/E_of_z(z',theta)

  distances(z, theta):
    Dc = comoving_distance(z,theta)
    DM = f_DM(Dc)        # typiquement DM=(1+z)*DA ; à plat: DM=Dc
    DH = c/(H0*E_of_z(z))
    DV = f_DV(DM, DH, z) # combinaison standard
    return (DM, DH, DV)

  mu_theory(z, theta):
    # mu = 5 log10( DL / 10pc ) + M (ou MB absorbé)
    DL = (1+z) * DM(z,theta) * c/H0  (selon conventions)
    return 5*log10(DL) + mu0(theta)  # mu0 encapsule MB/H0 selon votre choix

LIKELIHOODS:
  chi2_SN(theta):
    mu_th = [mu_theory(z_i,theta) for i in SN]
    r = mu_obs - mu_th
    return r^T * inv(C_mu) * r

  chi2_CC(theta):
    H_th = [H0*E_of_z(z_j,theta) for j in CC]
    r = H_obs - H_th
    return r^T * inv(C_CC) * r       # ou somme r^2/sigma^2

  chi2_BAO(theta):
    # construit y_th à même format que y_obs (DM/rs, DH/rs, DV/rs, ...)
    y_th = []
    for each BAO datum at z_k:
      (DM,DH,DV) = distances(z_k,theta)
      y_th.append( build_vector(DM,DH,DV, rs_eff, datum_type) )
    r = y_obs - y_th
    return r^T * inv(C_bao) * r

  chi2_total(theta):
    return chi2_SN(theta) + chi2_CC(theta) + chi2_BAO(theta) + prior_penalty(theta)

OPTIMIZATION / INFERENCE:
  initialize theta0 within bounds
  theta_best = argmin_theta chi2_total(theta)  # (Nelder-Mead / L-BFGS-B / etc.)
  compute cov_theta ~ inverse Hessian or MCMC (option)

DIAGNOSTICS / ROBUSTNESS (as described for Annexe B):
  - redo fit with:
      A_b = 0 (no-bump)
      gating off/on
      leave-one-survey-out BAO (ou retrait d’un point)
      variations CC (SPS, sous-échantillon, covariance)
  - compute Δchi2 between baselines
  - compute residuals per probe (SN/CC/BAO) and per observable (DM/rs, DH/rs, DV/rs)
  - check stability of (H0, rs_eff) and of shape parameters
  - optional: fit SN+CC then predict BAO (train/test)

OUTPUTS:
  theta_best, uncertainties
  chi2_SN, chi2_CC, chi2_BAO, chi2_total
  curves: E(z), H(z), HD(z), distances, residual plots
  robustness table: Δchi2 across ablations
